<a id="Contents"></a>
# Reasoning Engine Basics

This notebook provides a basic introduction to the reasoning engine framework and the Reasoning Engine Intermediate Language (REIL). 

* [Introduction](#Introduction)
* [Model Definition and Model Checking](#ModelChecking): Basic model definition using REIL and SMT-based verification/synthesis 
* ['Hybrid' Models](#HybridModels): Defining models that combine Boolean and numerical variables
* [Synchronous vs Asynchronous Updates](#SyncVsAsync): Examples of defining synchronous and asynchronous dynamical systems
* [Model Enumeration](#Enumeration): When synthesizing a single satisfying model is not enough
* [System and Path Variables](#VariableScope): Examples of different variable scopes
* [Higher-level DSLs](#DSLs): Examples of using REIL to define richer, domain-specific languages
* [Notes](#Notes): Some additional notes

In [1]:
#load @"../REInteractiveAPI/ReLoad.fsx"
open ReasoningEngine

The below script needs to be able to find the current output cell; this is an easy method to get it.

Loading the Reasoning Engine (RE)...


<a id="Introduction"></a>
## Introduction

A Reasoning Engine ``model`` is described by a number of discrete state variables, which could be of ``int`` (int), ``nat`` (non-negative integer) or ``bool`` (Boolean) type. Each variable can be either a ``system``, ``path`` or ``state`` variable. Path variables are replicated for each trajectory of the system that is considered as part of the analysis. State variables are replicated for each experiment and at every discrete time step and system variable are not replicated. Constraints are defined over the different variables of the system.


<a id="ModelChecking"></a>
## Model Definition and Model Checking
For our first model, we consider a simple system with only a single state variable ``x``, which is of type ``int``. We specify the model using the REIL language, load it and check if solutions exist using the SMT solver Z3.

In [2]:
"""
state int x;
"""
|> ReilAPI.Load
|> ReilAPI.CheckAndPrint

Solution(s) found

Using the SMT solver we find that solutions exist, meaning that a valid assignment of all system variables can be found. However, this is not very interesting because our system does not specify any constraints or even system dynamics yet. Let's fix that next by specifying that our system is a simple counter, where the value of ``x`` increases at every step.  

We will also specify some constraints about the executions of our counter. We consider an execution (also reffered to as a trajectory or path) of the system called ``test`` and specify that in this execution the value of the counter was initially ``0`` and is above ``0`` at step ``10``. 

In [3]:
"""
state int x;
update p[k].x := p[k-1].x + 1;
#test[0].x = 0;
#test[10].x > 0;
"""
|> ReilAPI.Load
|> ReilAPI.Check
|> TrajVis.PlotSolutionTrajectories

Next, we relax the constraints on our counter a bit. Instead of specifying the initial state, we simply require that the value of ``x`` is above ``20`` at step ``10``. We synthesize a solution, where a suitable initial state has been selected by the solver to ensure that the constraint is satisfied.

In [4]:
"""
state int x;
update p[k].x := p[k-1].x + 1;
#test[10].x > 20;
"""
|> ReilAPI.Load
|> ReilAPI.Check
|> TrajVis.PlotSolutionTrajectories

Multiple executions of the same system but with different constraints can be specified and considered as part of the synthesis problem. Here, we require that for the same counter system one trajectory ``test1`` is above ``20`` and another is below ``5`` at step ``10``. Once again, suitable and different initial states are synthesized for the two trajectories to satisfy the constraints.

In [5]:
"""
state int x;
update p[k].x := p[k-1].x + 1;
#test1[10].x > 20;
#test2[10].x < 5;
"""
|> ReilAPI.Load
|> ReilAPI.Check
|> TrajVis.PlotSolutionTrajectories

<a id="HybridModels"></a>
## 'Hybrid' Models

Although the Reasoning Engine currently supports only discrete models, variables can be of types ``int`` (or ``nat``) and ``bool``. This allows for the construction of 'hybrid' models that combine logical and numerical dynamics. 

In the following, we develop a basic temperature control system model. We assume that the temperature increases by one degree per time step whenever the heating is on but decreases when the heating is off. To capture this, we use two ``state`` variables: an ``int`` variable ``temperature`` (since no continuous quantities are currently supported) and a ``bool`` ``heasIsOn``. We define the system dynamics such that the heating is switch on whenever the temperature drops below ``18`` degrees. We are interested in checking whether it is possible to reach temperature below ``18`` despite our temperature controller, which turns out to be possible as illustrated by the trajectories synthesized by the Reasoning Engine.

In [6]:
"""
unique state int temperature;
unique state bool heatIsOn;
update 
    p[k].temperature := if (p[k-1].heatIsOn) then (p[k-1].temperature + 1) else (p[k-1].temperature - 1),
    p[k].heatIsOn := p[k-1].temperature < 18;

#test[0].temperature = 20;
#test[10].temperature < 18;
"""
|> ReilAPI.Load
|> ReilAPI.Check
|> TrajVis.PlotSolutionTrajectories

<a id="SyncVsAsync"></a>
## Synchronous vs Asynchronous Updates

The temperature controller example above introduced update rules for two separate variables. As defined for the temperature controller this specifies that both variables are updated synchronously, which generally results in deterministic updates (unless there are other sources of non-determinism). The same update rules are used to define the example below, where state variables ``x`` and ``y`` are updated synchronously at each time step. No solutions are found in this case because the value reached at step ``10`` (for both ``x`` and ``y``) cannot be ``5``.

In [7]:
"""
unique state int x;
unique state int y;
update 
    p[k].x := p[k-1].x + 1,
    p[k].y := p[k-1].y + 1;
#test[0].x = 0;
#test[0].y = 0;
#test[10].x = 5;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 10
|> TrajVis.PlotSolutionTrajectories

No solutions found

The Reasoning Engine also supports asynchronous update rule definitions. The example below is similar to the one above but now either of the two update rules can be triggered asynchronously at each time step. Here, we find solutions for the same constraints because different interleavings of the two update rules can result in trajectories, where ``x`` reaches the value of ``5``.  

In [8]:
"""
unique state int x;
unique state int y;
update p[k].x := p[k-1].x + 1;
update p[k].y := p[k-1].y + 1;
#test[0].x = 0;
#test[0].y = 0;
#test[10].x = 5;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 10
|> TrajVis.PlotSolutionTrajectories

<a id="Enumeration"></a>
## Model Enumeration

In the examples so far, we were only interested in synthesizing a single model of the dynamical system satisfying all constraints. If we assume that the constraints represent some behavior of the system we have observer, in general there might be many possible models capable of reproducing this behavior. Selecting only a single solution is a common problem in the modeling of physical system, which can bias the results and conclusions by introducing hidden assumptions (e.g. why is the first model better). To address this problem, the Reasoning Engine provides functionality for enumerating multiple models consistent with the constraints. 

To illustrate this, we again consider the counter system. We modify the system definition by specifying that the variable ``x`` is ``unique``. Defining variables as unique provides a mechanisms for specifying when is one solution different from another (e.g. at least one unique variable must be different). Similarly, non-unique variables are not considered as part of the enumeration, thus reducing the number of possible solutions. 

Enumerating 5 different solutions for our counter system reveals different ways of achieving the specification and reaching a value above ``20`` at step ``10``.

In [9]:
"""
unique state int x;
update p[k].x := p[k-1].x + 1;
#test[10].x > 20;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 5
|> TrajVis.PlotSolutionTrajectories

<a id="VariableScope"></a>
# System and path variables

In the examples so far we only considered ``state`` variables (i.e. variables that change along the executions of the system). Besides ``state`` variables, the Reasoning Engine also handles variables with ``system`` or ``path`` scope. 

In the example below, we extend the counter system, so that it can count by an arbitrary ``increment``. In this model, the increment is a system properties, so all trajectories of the system would increase by the same amount at each step. We also tighten the constraints again, by specifying that our counter starts at ``0`` and enumerate solutions. 

While the state variable ``x`` is unique, trajectories of this model are in fact deterministic (and here start from ``0``) once the ``increment`` is selected. Therefore, the different solutions represent different possible choices of the unique ``increment`` variable. While we are enumerating up to ``100`` different solutions, only three models are synthesized, indicating that no other models are consistent with the constraints of achieving a value between ``10`` and ``50`` at step ``10``.

In [10]:
"""
unique state int x;
unique system nat increment;
update p[k].x := p[k-1].x + increment;
#test[0].x = 0;
#test[10].x > 10;
#test[10].x < 50;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 100
|> TrajVis.PlotSolutionTrajectories


To illustrate the difference between ``system`` and ``path`` variables, we consider the problem of synthesizing a counter that can reach the value of either ``10`` or ``20`` in two different executions (``test1`` and ``test1``), both of which start at ``0``.

In [11]:
"""
unique state int x;
unique system nat increment;
update p[k].x := p[k-1].x + increment;
#test1[0].x = 0;
#test2[0].x = 0;
#test1[10].x = 10;
#test2[10].x = 20;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 100
|> TrajVis.PlotSolutionTrajectories


No solutions found

We verify that this problem is unsatisfiable and no solutions exist. This is expected, since the same ``system`` increment cannot produce two different trajectories. In the example below we modify the model so that ``increment`` is now a ``path`` variable. This allows for a distinct value of ``increment`` to be synthesized for each of the two executions and the problem becomes satisfiable. Notice that only a single solution is found despite the enumeration. Indeed ``#test1.increment = 1`` and ``#test2.increment = 2`` is the only possible variable assignment that is consistent with the specification.

In [12]:
"""
unique state int x;
unique path nat increment;
update p[k].x := p[k-1].x + p.increment;
#test1[0].x = 0;
#test2[0].x = 0;
#test1[10].x = 10;
#test2[10].x = 20;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 100
|> TrajVis.PlotSolutionTrajectories


In [13]:
"""
unique state int x;
unique state int y;
update p[k].x := p[k-1].x + p[k].y;
#test[0].x = 0;
#test[0].y = 1;
#test[10].x > 0;
"""
|> ReilAPI.Load
|> ReilAPI.Enumerate 100
|> TrajVis.PlotSolutionTrajectories

<a id="DSLs"></a>
## Higher-level DSLs

The purpose of REIL is to provide a basic intermediate language that could be a compilation target for a variety of DSLs. With this approach, the low-level details of encoding problems into SMT, calling the solver (in this case Z3) and processing the solutions are handled by the Reasoning Engine. Furthermore, this approach allows for additional tactics, problem transformations, simplifications and reasoning strategies to be implemented as part of the Reasoning Engine and reused when reasoning about models defined using higher-level DSLs. The approach has been used to define several biological DSLs compiling to REIL in other projects.

In the following we illustrate this idea by creating a simple 'embedded DSL' for Boolean networks. These networks are simple models of genetic interaction. At each state, each gene is either active (true) or inactive (false). Interactions between the genes define the next state of the system. For simplicity, here we consider only synchronous networks with AND-type regulation. We also implement only very limited constraints (e.g. to capture experimental biological observations). Instead of producing REIL text, the example below constructs directly the F# Reasoning Engine model as an illustration of possible programmatic modeling.

In [14]:
open Microsoft.Research.ReasoningEngine.Model
open Microsoft.Research.ReasoningEngine.Var
open Microsoft.Research.ReasoningEngine.Dynamics
open Microsoft.Research.ReasoningEngine.Constraint

type Gene = Gene of string

type Interaction = 
    | Activates of (Gene * Gene) //Source * Target
    | Represses of (Gene * Gene)
    
type Observation = 
    | Active of   (string * int * Gene) //experiment * gene * step
    | Inactive of (string * int * Gene)
    
type Network = 
    { genes        : Gene[]
      interactions : Interaction[]
      observations : Observation[]
    }
    static member Encode (network:Network) = 
        //define a state variable for each gene
        let sys = Array.fold (fun (acc:DSystem) (Gene g) -> acc.DeclareStateVar(g,Type.Bool)) DSystem.EmptySystem network.genes
        
        //add update rules based on the unteractions
        let updateRules = 
            network.interactions
            |> Array.map (fun interaction -> 
                match interaction with 
                | Activates (source, target) -> (true, source, target)
                | Represses (source, target) -> (false, source, target)
                )             
            |> Array.groupBy (fun (_,_,target) -> target)
            |> Array.map(fun (Gene target, regulators) -> 
                regulators
                |> Seq.map(fun (flag, Gene source, _) -> 
                    let v = 
                        AbsStateVar(-1, source) //for all paths, for all steps k, consider k-1
                        |> BVar
                        |> BTerm
                    if flag then v else Not v
                    )
                |> LAnd
                |> BExpr
                |> fun expr -> AssignmentRule.Create(target, expr) |> Assignment
                )
            
        let sys = {sys with updates = [Update.Create(None, updateRules)]}
        
        let model = Model.NewModel sys
        
        //add constraints
        network.observations
        |> Array.map(fun obs -> 
            match obs with 
            | Active (exp, t, g) -> (true, exp, t, g)
            | Inactive (exp, t, g) -> (false, exp, t, g)
            )
        |> Array.map(fun (flag, exp, t, Gene g) -> 
            let v = 
                StateVar(exp, t, g) //for all paths, for all steps k, consider k-1
                |> BVar
                |> BTerm
            if flag then v else Not v            
            )
        |> Array.fold (fun acc cst -> 
                {acc with constraints = acc.constraints.AddObservation(cst, "Constraint")}
            ) model

        

Next, we use our 'embedded DSL' to define a simple oscillating gene network based on the [Repressilator design](https://en.wikipedia.org/wiki/Repressilator).

In [15]:
let A, B, C = Gene "A", Gene "B", Gene "C"

{ genes = [|A; B; C|]
  interactions = 
      [| Represses (A, B)
         Represses (B, C)
         Represses (C, A)
      |]
  observations = 
      [| Active("Experiment1", 0, A)
         Inactive("Experiment1", 0, B)
         Inactive("Experiment1", 0, C)
         Inactive("Experiment1", 10, A)
      |]
}
|> Network.Encode
|> ReilAPI.Enumerate 100
|> TrajVis.PlotSolutionTrajectories

<a id="Notes"></a>
## Notes

* **Notation:** In the examples above, we used ``update p[k].x := p[k-1].x + 1`` to define the system dynamics. This can be read as defining an update rule, where for all paths ``p``, for all time steps ``k`` the value of state variable ``x`` at time ``k`` of path ``p`` is determined from the value of ``x`` at the previous time step ``p[k-1].x``. Path variables do not have a time step, so can be referenced by just the generic path ``p`` in update rules (e.g. ``update p[k].x := p[k-1].x + p.increment`` for ``path`` variable ``increment``). Since system variables are not associated with a path, there is no additional scoping (e.g. ``update p[k].x := p[k-1].x + increment`` for ``system`` variable ``increment``).

* **Undefined Dynamics:** Without any update rules, currently the problems become unsatisfiable. Alternatively, not-deterministic updates (any next state is possible for all variables) or constant dynamics (e.g. ``p.[k].x = p.[k-1].x``) could be assumed in this edge case. If an update is not specified for a given variable but update rules are otherwise specified, the variable is assumed to retain its value from the previous step (see example below).

* **Delays:** Technically, the update rules allow for arbitrary delays to be specified (e.g. ``update p[k].x := p.[k-2].x + 1``). However, such dynamics cause problems in the current implementation, where the first update rule is applied at time-step ``1``. Rules that involve the current time-step (e.g. ``update p[k].x = p[k-1].x+p.[k].y``) are also allowed as in the following.

* **Extensions:** There is additional, undocumented functionality implemented in the Reasoning Engine, some of which is purely experimental. For example, the REIL language supports cardinality constraints, which are useful for example to limit the number of Boolean choice variables that are set to true. There is also an experimental interface to [Infer.Net](https://dotnet.github.io/infer/), which could be used as an probabilistic reasoning strategy and an alternative to the SMT-based reasoning.